In [1]:
import numpy as np
import tbmodels
import h5py

In [2]:
def apply_efield(h0,efield,z_pos):
    h = h0.copy()
    for i in np.arange(h0.shape[0]):
        h[i,i] = h0[i,i]+efield*z_pos[i]
    return h

def find_efermi_bands(eigvals_all, nfilling):
    vbm = eigvals_all[:,nfilling-1].max()    
    cbm = eigvals_all[:,nfilling].min()
    return vbm, cbm, (vbm+cbm)/2

def find_splitted_cbm_edges(eigvals_all, nfilling):
    cbm1 = eigvals_all[:,nfilling].min()
    # find the corresponding index of k
    k_cbm1 = np.argmin(eigvals_all[:,nfilling]) 

    cbm2 = eigvals_all[:,nfilling+1].min()
    # find the corresponding index of k
    k_cbm2 = np.argmin(eigvals_all[:,nfilling+1])
    return k_cbm1, k_cbm2, cbm1, cbm2

In [3]:
nwann = 56
nfilling = 36

wpath = f"../tb_model/model_o{nwann}/"

model = tbmodels.Model.from_wannier_files(
            hr_file = f'{wpath}/wte2.o{nwann}_hr.dat',
            xyz_file = f'{wpath}/wte2.o{nwann}_centres.xyz',
            win_file = f'{wpath}/wte2.o{nwann}.win')

# apply on-site electric fields to the wannier orbitals
c = 6.576*4.0316*0.529177
z_pos = model.pos[:,2]*c

z_pos_sym = z_pos.copy()
for iz in np.arange(z_pos.shape[0]):
    z_pos_sym[iz] = 0.5*(z_pos[iz]+z_pos[iz+(-1)**(iz%2)])

In [4]:
# generate a list of k points along the -X-Gamma-X path
nk = 101
kx_list = np.linspace(-0.5,0.5,nk)
k_list = [[kx,0,0] for kx in kx_list]

In [6]:
# generate the band structure on a given k-path for the following efields
efields = [-0.5, -0.35, -0.3, -0.2, -0.15, -0.1, -0.05, 0.0, 0.05, 0.1, 0.15, 0.2, 0.3, 0.35, 0.5]

with h5py.File("./data/bulk_bands_efields.h5", 'w') as f:
    f.create_dataset("kx_list", data=kx_list)
    for efield in efields:
        h0_k = model.hamilton(k_list)
        h_k = [apply_efield(h0,efield,z_pos) for h0 in h0_k]
        eigvals, eigvecs = np.linalg.eigh(h_k)
        vbm, cbm, efermi = find_efermi_bands(eigvals, nfilling)
        k_cbm1, k_cbm2, cbm1, cbm2 = find_splitted_cbm_edges(eigvals, nfilling)
        if efield == -0.5:
            print("efield, efermi, cbm, cbm1, cbm2:")
        print(efield, efermi, cbm, cbm1, cbm2)
        # save the eigvals and eigvecs to the file
        f.create_dataset(f"e{efield}/eigvals", data=eigvals-efermi)
        f.create_dataset(f"e{efield}/eigvecs", data=eigvecs)
        f.create_dataset(f"e{efield}/vbm", data=vbm-efermi)
        f.create_dataset(f"e{efield}/cbm", data=cbm-efermi)
        f.create_dataset(f"e{efield}/efermi", data=efermi)
        f.create_dataset(f"e{efield}/cbm1", data=cbm1-efermi)
        f.create_dataset(f"e{efield}/cbm2", data=cbm2-efermi)
        f.create_dataset(f"e{efield}/k_cbm1_index", data=k_cbm1)
        f.create_dataset(f"e{efield}/k_cbm2_index", data=k_cbm2)


efield, efermi, cbm, cbm1, cbm2:
-0.5 -1.2299713816153979 -1.2147189931206148 -1.2147189931206148 -0.9330698268753133
-0.35 -0.16649843697765065 -0.1714154056118108 -0.1714154056118108 0.06805219830348028
-0.3 0.1939483756608003 0.18950477240832025 0.18950477240832025 0.4021101093658835
-0.2 0.9211341365594877 0.9259214297531912 0.9259214297531912 1.071853569475122
-0.15 1.286440163558138 1.2982165976391673 1.2982165976391673 1.4076499201969281
-0.1 1.6521642731345825 1.6720483787734672 1.6720483787734672 1.7441598602081116
-0.05 2.018087194156686 2.0469832897439644 2.0469832897439644 2.0814810271448194
0.0 2.3825020706388162 2.419610652255581 2.419610652255581 2.4202490675936534
0.05 2.729710377689674 2.758534524815379 2.758534524815379 2.7941288135081814
0.1 3.0757152781904633 3.0957604281159155 3.0957604281159155 3.1690720636197542
0.15 3.4219315797698693 3.4341151220402466 3.4341151220402466 3.5448397206870372
0.2 3.768758576073611 3.774066322192018 3.774066322192018 3.921340622113